In [1]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import ipywidgets as widgets
import serial
import time
from picoscope import ps5000a
from picosdk.discover import find_all_units
import serial.tools.list_ports as port_list
import chipwhisperer as cw

## Program target with Design_Start soft core and AES firmware ##

Convenience functions

In [2]:
# some convenience functions:
def reset_fpga():
    # resets the full CW305 FPGA
    ftarget.fpga_write(3, [1])
    ftarget.fpga_write(3, [0])

def reset_arm_target():
    # resets only the Arm DesignStart core within the CW305 FPGA
    ftarget.fpga_write(2, [1])
    ftarget.fpga_write(2, [0])

def reset_flush():
    reset_arm_target(ftarget)
    ftarget.flush()

Load bitstream to FPGA

In [3]:
bitstream = r"../../pv_CW305_DesingStart/hardware/pv_CW305_DesignStart/pv_CW305_DesignStart.bit"
ftarget = cw.target(None, cw.targets.CW305, bsfile=bitstream, force=True)
print(ftarget.is_programmed())
reset_fpga()
reset_arm_target()
print(ftarget.get_fpga_buildtime())


True
0/0/2000, 00:00


## Setup target platform ##

In [4]:
# set clock:
ftarget.vccint_set(1.0)
# we only need PLL1:
ftarget.pll.pll_enable_set(True)            # enable PLL chip
ftarget.pll.pll_outenable_set(False, 0)     # disable PLL 0
ftarget.pll.pll_outenable_set(True, 1)      # enable PLL 1 
ftarget.pll.pll_outenable_set(False, 2)     # disable PLL 2

# run at 20 MHz:
ftarget.pll.pll_outfreq_set(20E6, 1)


##  Setup picoscope ##

In [5]:
scopes = find_all_units()
for scope in scopes:
    print("Working with:")
    print(scope.info)
    scope.close()
ports = list(port_list.comports())
for p in ports:
    print (p)

ps = ps5000a.PS5000a()

# Since target runnning at 20 MHz then AES requires from trigger 3.225 uS
obs_duration = 3.225E-6
# Sample at least 1260 points within that window
sampling_interval = obs_duration / 1260
# Configure timebase
(actualSamplingInterval, nSamples, maxSamples) = ps.setSamplingInterval(sampling_interval, obs_duration)
print("\nNsamples : ", nSamples)
print("Sampling interval = %f us" % (actualSamplingInterval*nSamples*1E6))

# 50mV range on channel A, AC coupled, 20 MHz BW limit
ps.setChannel('A', 'AC', 0.05, 0.0, enabled=True, BWLimited=True)
# Channel B is trigger
ps.setChannel('B', 'DC', 10.0, 0.0, enabled=True)
ps.setSimpleTrigger('B', 2.0, 'Rising', timeout_ms=2000, enabled=True)

Working with:
UnitInfo(driver=<picosdk.ps5000a.Ps5000alib object at 0x7f8c74878490>, variant=b'5244D', serial=b'KU687/0175')
/dev/ttyS4 - n/a

Nsamples :  1612
Sampling interval = 3.224000 us


## Run capture  ##

In [6]:
# capture functions
def pico_capture():
    # Arm the picoscope
    ps.runBlock()
    time.sleep(0.05)
    # Trigger the encryption on Target
    ftarget.fpga_write(ftarget.REG_USER_LED, [0x01])
    ftarget.usb_trigger_toggle()
    ps.waitReady()
    # Capture the trace 
    data = ps.getDataV('A', nSamples, returnOverflow=False)
    return data


Capture loop

In [7]:
from Crypto.Cipher import AES
from chipwhisperer.common.traces import Trace

reset_arm_target()
ktp = cw.ktp.Basic()
# Initialize cipher to verify DUT result:
key, plaintext = ktp.next()
cipher = AES.new(bytes(key), AES.MODE_ECB)
print("Key: ", [hex(el) for el in key])
print("Plaintext: ", [hex(el) for el in plaintext])

output_len = 16

# Dummy capture call due to bug of using AC coupling
pico_capture()

# Load key
ftarget.simpleserial_write('k', key)
# Check if the value has been written correctly 
key_written = ftarget.fpga_read(ftarget.REG_CRYPT_KEY, output_len)
print("\nValue in REG_CRYPT_KEY: ", [hex(el) for el in key_written])

# Load plaintext
ftarget.simpleserial_write('p', plaintext)
# Check if the value has been written correctly
text_written = ftarget.fpga_read(ftarget.REG_CRYPT_TEXTIN, output_len)
print("Value in REG_CRYPT_TEXTIN: ", [hex(el) for el in text_written])

# Launch capture
wave = pico_capture()

# Read response
response = ftarget.simpleserial_read('r', output_len, ack=True)
print("\nCiphertext: ", [hex(el) for el in response])
print("Expected : ", [hex(el) for el in cipher.encrypt(bytes(plaintext))])

# Values read from registers
print("\nREG_CRYPT_CIPHEROUT: ", [hex(el) for el in response])
print("REG_CRYPT_TEXTIN: ", [hex(el) for el in text_written])
print("REG_CRYPT_KEY: ", [hex(el) for el in key_written])

# Disconnect for all devices
#ftarget.dis()

#assert (list(response) == list(cipher.encrypt(bytes(plaintext)))), "Incorrect encryption result!\nGot {}\nExp {}\n".format(list(response), list(plaintext))
    

Key:  ['0x2b', '0x7e', '0x15', '0x16', '0x28', '0xae', '0xd2', '0xa6', '0xab', '0xf7', '0x15', '0x88', '0x9', '0xcf', '0x4f', '0x3c']
Plaintext:  ['0x67', '0x75', '0xf6', '0x74', '0x5a', '0xfe', '0x9', '0xd3', '0xc4', '0xf7', '0x2c', '0x6f', '0x53', '0x8b', '0x71', '0x4b']

Value in REG_CRYPT_KEY:  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']
Value in REG_CRYPT_TEXTIN:  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']

Ciphertext:  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']
Expected :  ['0x31', '0xc6', '0xad', '0x97', '0xa8', '0xce', '0xa5', '0x88', '0x38', '0xe1', '0x8a', '0x6f', '0x80', '0x88', '0xb4', '0x27']

REG_CRYPT_CIPHEROUT:  ['0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']
REG_CRYPT_TEXTIN:  ['0x0', '0x0', '0x0', '

Plotting trace

In [8]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool

output_notebook()
p = figure(plot_width=800)

xrange = range(len(wave))
p.line(xrange, wave, line_color="red")
show(p)

Loading BokehJS ...

In [9]:
import os
#os.system('jupyter nbconvert --to html capture_PowerTrace_CW305_DesignStart.ipynb')

[NbConvertApp] Converting notebook capture_PowerTrace_CW305_DesignStart.ipynb to html
[NbConvertApp] Writing 339988 bytes to capture_PowerTrace_CW305_DesignStart.html


0